In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('HistoricalQuotes.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    253 non-null    object 
 1   close   253 non-null    float64
 2   volume  253 non-null    object 
 3   open    253 non-null    float64
 4   high    253 non-null    float64
 5   low     253 non-null    float64
dtypes: float64(4), object(2)
memory usage: 12.0+ KB


In [8]:
df = pd.read_csv("HistoricalQuotes.csv",
                 parse_dates=['date'])
# date format is MM - DD - YYYY

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    253 non-null    datetime64[ns]
 1   close   253 non-null    float64       
 2   volume  253 non-null    object        
 3   open    253 non-null    float64       
 4   high    253 non-null    float64       
 5   low     253 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 12.0+ KB


In [10]:
df.shape

(253, 6)

In [11]:
df.set_index('date',inplace=True)

In [13]:
df.head()

,close,volume,open,high,low
date,,,,,
2019-08-10,200.99,"24,619,446",201.23,202.760,199.29
2019-08-09,200.99,24619750.0000,201.30,202.760,199.29
2019-08-08,203.43,27009520.0000,200.20,203.530,199.39
2019-08-07,199.04,33364400.0000,195.41,199.560,193.82
2019-08-06,197.00,35824790.0000,196.31,198.067,194.04


In [14]:
df.sort_index(inplace=True)

In [15]:
df.head()

,close,volume,open,high,low
date,,,,,
2018-08-09,208.88,23389530.0000,207.280,209.780,207.20
2018-08-10,207.53,24592460.0000,207.360,209.100,206.67
2018-08-13,208.87,25864510.0000,207.700,210.952,207.70
2018-08-14,209.75,20679270.0000,210.155,210.560,208.26
2018-08-15,210.24,28595230.0000,209.220,210.740,208.33


# Missing value filling
freq in B days

In [17]:
df = df.asfreq('B')

In [18]:
df.isna().sum()

close     10
volume    10
open      10
high      10
low       10
dtype: int64

In [19]:
df.fillna(method='ffill',inplace=True)

In [20]:
df.isna().sum()

close     0
volume    0
open      0
high      0
low       0
dtype: int64

In [22]:
df.shape

(262, 6)

# Create log columns based on close

In [23]:
df['lag1'] = df['close'].shift(1)
df['lag2'] = df['close'].shift(2)
df['lag3'] = df['close'].shift(3)
df['lag4'] = df['close'].shift(4)
df['lag5'] = df['close'].shift(5)

In [24]:
df.head()

,close,volume,open,high,low,lag1,lag2,lag3,lag4,lag5
date,,,,,,,,,,
2018-08-09,208.88,23389530.0000,207.280,209.780,207.20,NaN,NaN,NaN,NaN,NaN
2018-08-10,207.53,24592460.0000,207.360,209.100,206.67,208.88,NaN,NaN,NaN,NaN
2018-08-13,208.87,25864510.0000,207.700,210.952,207.70,207.53,208.88,NaN,NaN,NaN
2018-08-14,209.75,20679270.0000,210.155,210.560,208.26,208.87,207.53,208.88,NaN,NaN
2018-08-15,210.24,28595230.0000,209.220,210.740,208.33,209.75,208.87,207.53,208.88,NaN


In [25]:
df.dropna(inplace=True)

In [26]:
df.shape

(257, 10)

In [27]:
df.head()

,close,volume,open,high,low,lag1,lag2,lag3,lag4,lag5
date,,,,,,,,,,
2018-08-16,213.32,27595310.0000,211.75,213.8121,211.470,210.24,209.75,208.87,207.53,208.88
2018-08-17,217.58,35034410.0000,213.44,217.9500,213.160,213.32,210.24,209.75,208.87,207.53
2018-08-20,215.46,30149410.0000,218.10,219.1800,215.110,217.58,213.32,210.24,209.75,208.87
2018-08-21,215.04,25682700.0000,216.80,217.1900,214.025,215.46,217.58,213.32,210.24,209.75
2018-08-22,215.05,18965930.0000,214.10,216.3600,213.840,215.04,215.46,217.58,213.32,210.24


# X&Y 

In [28]:
X = df.loc[:,['lag1','lag2','lag3','lag4','lag5']]
y = df.loc[:,'close']

In [30]:
X.shape,y.shape

((257, 5), (257,))

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train,X_test,y_train,y_test = train_test_split(X,y,
                                                 test_size=0.1,
                                                 random_state=7)

In [33]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((231, 5), (26, 5), (231,), (26,))

# Elastic Net

In [35]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [36]:
en = ElasticNet(random_state=7)

In [37]:
en.fit(X_train,y_train)

ElasticNet(random_state=7)

In [38]:
y_pred = en.predict(X_test)

In [39]:
y_pred.shape,y_test.shape

((26,), (26,))

In [40]:
mean_absolute_error(y_test, y_pred)

2.828605621122141